In [ ]:
!py -m pip install tensorflow pandas numpy matplotlib

In [ ]:
import pandas as pd
import requests
import re
import numpy as np

def url_getter(year):
  return "https://www.pro-football-reference.com/draft/" + str(year) + "-combine.htm"

def combine_draft_scraper(start_year, end_year):
  for i in range(start_year, end_year+1):
    request = requests.get(url_getter(i))
    try:
      raw = pd.read_html(request.text)
    except:
      print(request.text)
      break
    df = raw[0]
    df = df.drop(['College'], axis=1)
    temp_pick_num = []
    temp_pick_year = []
    for index, t in df.iterrows():
      if t['Player'] == "Player":
        df.drop(index=index, axis=0, inplace=True)
      else:
        # print(type(t["Drafted (tm/rnd/yr)"]))
        if not t.isna()[t.size-1] == True:
          year = ''.join(a for a in str(t["Drafted (tm/rnd/yr)"]).split(" / ")[1] if a.isdigit())
          temp_pick_num.append(year)
          print(str(t["Drafted (tm/rnd/yr)"]).split("/")[3])
          temp_pick_year.append(int(str(t["Drafted (tm/rnd/yr)"]).split("/")[3]))
        else:
          temp_pick_num.append(str(0))
          temp_pick_year.append(str(-1))
        htstr = str(t["Ht"]).split('-')
        if not t.isna()['Ht']:
          df.loc[index, 'Ht'] = (int(htstr[0]) * 12) + int(htstr[1])
    df = df.drop(columns=['Drafted (tm/rnd/yr)'])
    # print(temp_pick_num);
    df["Pick Number"] = pd.Series(temp_pick_num)
    df["Pick Year"] = pd.Series(temp_pick_year)
    df.to_csv(str(i) + ".csv")

combine_draft_scraper(2000, 2020)



In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.impute import KNNImputer
df = pd.read_csv("2000.csv")

#train on 80 percent of data
for i in range(2001, 2011):
  df = pd.concat([df, pd.read_csv(str(i) + ".csv")])

# df_validate = pd.read_csv("2011.csv");

# for i in range(2012, 2022):
#   df_validate = pd.concat([df, pd.read_csv(str(i) + ".csv")])

df = df[:2000]
# df_validate = df_validate[:2000]

for index, row in df.iterrows():
  if (row["Pick Number"] > 7) or (row["Pick Number"] <= 0):
    df = df.drop(, axis=0)
    print("test")
df_values = df["Pick Number"];

#remove useless data
df = df.drop(['School'], axis=1)
df = df.drop(['Pos'], axis=1)
df = df.drop(['Pick Year'], axis=1)
df = df.drop(['Player'], axis=1)
df = df.drop(['Unnamed: 0'], axis=1)

# df_validate = df_validate.drop(['School'], axis=1)
# df_validate = df_validate.drop(['Pos'], axis=1)
# df_validate = df_validate.drop(['Pick Year'], axis=1)
# df_validate = df_validate.drop(['Player'], axis=1)
# df_validate = df_validate.drop(['Unnamed: 0'], axis=1)

#no answers in training data
df = df.drop(['Pick Number'], axis=1)
# df_validate = df_validate.drop(['Pick Number'], axis=1)

#training
model = tf.keras.Sequential([
    tf.keras.layers.Normalization(axis=None, input_shape=(8,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(8)
])

df = KNNImputer(n_neighbors=5).fit_transform(df)
df = tf.convert_to_tensor(df, dtype=tf.float32)
# df_validate = KNNImputer(n_neighbors=5).fit_transform(df_validate)
# df_validate = tf.convert_to_tensor(df_validate, dtype=tf.float32)
# df_values = tf.convert_to_tensor(df_values, dtype=tf.float32)
# df_validate_values = tf.convert_to_tensor(df_validate_values, dtype=tf.float32)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(df, df_values, epochs=10)

In [ ]:
!pip install -U scikit-learn


In [ ]:
!rm -rf *